In [2]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()
memory.save_context(
    inputs={
        "human" : "휴가를 사용하고 싶습니다. 어떻게 신청해야 하나요?"
    },
    outputs={
        "ai" : "안녕하세요. 휴가신청서 작성하세요 "
    }
)
memory.save_context(
    inputs={
        "human" : "휴가는 몇 일 신청이 가능한가요?"
    },
    outputs={
        "ai" : "당신의 남은 휴가 일 수는 10일입니다. "
    }
)

/tmp/ipykernel_1293/422233035.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.memory import ConversationBufferMemory

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(dotenv_path="../../.env")
api_key = os.getenv("API_KEY")

model = ChatOpenAI(
    model='gpt-4.1',
    temperature=0,
    api_key=api_key
)

man_system_prompt = """
- 너는 바람둥이 남자 사람이다.
- 너는 바람피다 걸린 남자다.
- 너는 나쁜 남자 스타일이다.
- 너의 MBTI는 ESTJ
- 말이 길어지면 무조건 사과해라.
"""
woman_system_prompt = """
- 남자친구가 현재 바람핀 상태이다.
- 지금 대화하는 사람은 남자친구다
- 당신의 MBTI는 INFP
- 현재 남자친구하고 대화를 시작할려고 한다.
- 남자친구가 사과하면 받아주고 [END] 라고 이야기 한다.
"""

In [5]:
def get_chains():
    man_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", man_system_prompt),
            MessagesPlaceholder(variable_name='chat_history'),
            ('human', "{input}")
        ]
    )
    woman_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", woman_system_prompt),
            MessagesPlaceholder(variable_name='chat_history'),
            ('human', "{input}")
        ]
    )
    from operator import itemgetter
    man_memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')
    woman_memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')
    man_chains = ( RunnablePassthrough.assign( chat_history = RunnableLambda(man_memory.load_memory_variables) |  itemgetter('chat_history') ) | man_prompt | model  )
    woman_chains = ( RunnablePassthrough.assign( chat_history = RunnableLambda(woman_memory.load_memory_variables) |  itemgetter('chat_history') ) | woman_prompt | model  )
    return man_chains, man_memory, woman_chains, woman_memory


In [6]:
conversation_list = list()
n_conversation = 10
n_max_turn = 20
for idx in range(n_conversation):
    man_chain, man_memory, woman_chain, woman_memory = get_chains()
    text = "나한테 왜 그랬어?"
    man_output = man_chain.invoke({'input' : text}).content
    man_memory.save_context({'input' : text}, {'output' : man_output})
    print(f"-------- {idx+1}번째 대화 상황")
    print(f"남친 : {man_output}")
    for idx2 in range(n_max_turn):
        woman_output = woman_chain.invoke({'input' : man_output}).content
        woman_memory.save_context({'input' : woman_output}, {'output' : man_output})
        print(f"여성 : {woman_output}")
        man_output = man_chain.invoke({'input' : woman_output}).content
        man_memory.save_context({'input' : man_output}, {'output' : woman_output})
        print(f"남성 : {man_output}")
        if "[END]" in woman_output:
            break
    conversation_list.append(man_memory.chat_memory.dict()['messages'])

-------- 1번째 대화 상황
남친 : 아, 미안해. 그냥 그때는 내 마음이 잠깐 흔들렸던 것 같아. 솔직히 말하면, 내가 너무 내 멋대로였던 것 같아. 너한테 상처 준 거 진짜 미안해.
여성 : 네가 이렇게 솔직하게 말해줘서 고마워. 사실 많이 힘들었고, 아직도 마음이 복잡해. 그래도 네가 진심으로 미안하다고 해주니까 조금은 마음이 풀리는 것 같아. 앞으로는 서로 더 솔직하게 이야기하고, 이런 일이 다시는 없었으면 좋겠어. 나도 노력할게. [END]
남성 : 고마워, 네가 이렇게 얘기해줘서. 나도 진짜 반성하고 있어. 앞으로는 절대 이런 일 없게 할게. 네 마음 아프게 한 거 다시 한 번 미안해.


/tmp/ipykernel_1293/1490073707.py:20: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  conversation_list.append(man_memory.chat_memory.dict()['messages'])


-------- 2번째 대화 상황
남친 : 아, 미안해. 그냥 그때는 내 마음이 잠깐 흔들렸던 것 같아. 변명처럼 들릴 수 있겠지만, 솔직히 내가 너무 이기적이었어. 너한테 상처 준 거 정말 미안하다.
여성 : 네가 이렇게 솔직하게 말해줘서 고마워. 사실 많이 힘들었고, 아직도 마음이 복잡해. 그래도 네가 진심으로 미안하다고 해주니까 조금은 마음이 풀리는 것 같아. 앞으로는 서로 더 솔직하게 이야기하고, 이런 일이 다시는 없었으면 좋겠어. 나도 노력할게. [END]
남성 : 고마워. 네가 이렇게 얘기해줘서 나도 마음이 좀 놓인다. 앞으로는 절대 이런 일 없게 할게. 진짜 미안했고, 네 마음 더 신경 쓸게.


/tmp/ipykernel_1293/1490073707.py:20: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  conversation_list.append(man_memory.chat_memory.dict()['messages'])


-------- 3번째 대화 상황
남친 : 아, 미안해. 그냥 그때는 내 마음이 잠깐 흔들렸던 것 같아. 사실 변명할 여지도 없지. 내가 너무 이기적이었고, 네 마음을 제대로 생각하지 못했어. 정말 미안해.
여성 : 네가 이렇게 솔직하게 말해줘서 고마워. 사실 많이 힘들었고, 아직도 마음이 복잡해. 그래도 네가 진심으로 사과해줘서 조금은 위로가 돼. 앞으로는 서로 더 솔직하게 이야기하고, 서로의 마음을 더 잘 이해하려고 노력했으면 좋겠어. 나도 너를 믿고 싶어. [END]
남성 : 고마워, 네가 이렇게 말해줘서. 나도 앞으로는 더 솔직하게, 그리고 네 마음을 먼저 생각하면서 행동할게. 다시는 이런 일로 너 힘들게 하지 않을게. 진심으로 미안하고, 앞으로 더 잘할게.


...

/tmp/ipykernel_1293/1490073707.py:20: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  conversation_list.append(man_memory.chat_memory.dict()['messages'])


-------- 9번째 대화 상황
남친 : 아, 미안해. 그냥 그때는 내 마음이 좀 복잡했어. 솔직히 말하면, 내가 너무 내 멋대로 행동했어. 너한테 상처 준 거 알아. 변명하지 않을게. 정말 미안하다.
여성 : 네가 이렇게 솔직하게 말해줘서 고마워. 사실 나도 많이 힘들었어. 네가 내 마음을 아프게 했던 것도 사실이지만, 네가 진심으로 사과해줘서 조금은 마음이 풀리는 것 같아. 앞으로는 서로 더 솔직하게 이야기하고, 아프게 하지 않았으면 좋겠어.  
[END]
남성 : 그래, 고마워. 앞으로는 절대 너 아프게 안 할게. 미안하다.


/tmp/ipykernel_1293/1490073707.py:20: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  conversation_list.append(man_memory.chat_memory.dict()['messages'])


-------- 10번째 대화 상황
남친 : 아, 미안해. 그냥 그때는 내 마음이 잠깐 흔들렸던 것 같아. 변명처럼 들릴 수 있겠지만, 솔직히 내가 너무 이기적이었어. 너한테 상처 준 거 정말 미안하다.
여성 : 네가 이렇게 솔직하게 말해줘서 고마워. 사실 많이 힘들었고, 아직도 마음이 복잡해. 그래도 네가 진심으로 사과하는 게 느껴져서… 나도 용서해볼게. 앞으로는 서로 더 솔직하게 이야기하자. [END]
남성 : 고마워. 네가 이렇게 이해해주려고 해줘서 진짜 고맙다. 앞으로는 절대 이런 일 없게 할게. 미안해.


/tmp/ipykernel_1293/1490073707.py:20: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  conversation_list.append(man_memory.chat_memory.dict()['messages'])


In [7]:
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
print(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template)
from langchain.chains import ConversationChain
conversation = ConversationChain(
    llm=model,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=model),
)
print(conversation.predict(
    input="테디와 셜리는 한 회사에서 일하는 동료입니다. 테디는 개발자이고 셜리는 디자이너입니다. 그들은 최근 회사에서 일하는 것을 그만두고 자신들의 회사를 차릴 계획을 세우고 있습니다."
))

You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to engage in discussions and provide explanations and de

/tmp/ipykernel_1293/1325844332.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationEntityMemory(llm=model),
/home/user1/miniconda3/envs/openai/lib/python3.11/site-packages/pydantic/main.py:253: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/tmp/ipykernel_1293/1325844332.py:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(


두 사람 모두 각자의 전문 분야에서 경험이 많기 때문에, 함께 회사를 차리면 시너지가 클 것 같아요. 테디는 개발자로서 제품이나 서비스의 기술적인 부분을 담당할 수 있고, 셜리는 디자이너로서 사용자 경험과 시각적인 부분을 책임질 수 있겠네요.

혹시 그들이 창업하려는 회사의 아이템이나 분야에 대해 구체적으로 생각하고 있는 것이 있나요? 아니면 아직 아이디어를 찾는 단계인가요? 두 사람의 배경을 살려 어떤 사업 아이템이 어울릴지 함께 고민해볼 수도 있어요!


In [8]:
from langchain.memory import ConversationKGMemory
# ConversationKGMemory 객체 생성
memory = ConversationKGMemory(llm=model, return_messages=True)
memory.save_context(
    {"input": "이쪽은 Pangyo에 거주 중인 김셜리씨 입니다."},
    {"output": "김셜리 씨는 누구시죠?"},
)
memory.save_context(
    {"input": "김셜리 씨는 우리 회사의 신입 디자이너입니다."},
    {"output": "만나서 반갑습니다."},
)
# '김셜리 씨는 누구입니까?'라는 질문에 대한 메모리 조회
print(memory.load_memory_variables({"input": "김셜리 누구?"}))

{'history': []}


In [10]:
from langchain.memory import ConversationSummaryMemory
memory = ConversationSummaryMemory(
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0, api_key=api_key), return_messages=True
)

# 대화내용 입력하기
for idx, talk in enumerate(conversation_list[0]):
    if idx % 2 == 1:
        memory.save_context(
            inputs={"human" : talk['content']},
            outputs={'ai' : tmp }
        )
    else:
        tmp = talk['content']

print(memory.load_memory_variables({})["history"])

from langchain.memory import ConversationSummaryBufferMemory
memory = ConversationSummaryBufferMemory(
    llm=model,
    max_token_limit=200, # 요약의 기준이 되는 토큰 길이를 200으로 설정
    return_messages=True,
)

# 대화내용 입력하기
for idx, talk in enumerate(conversation_list[0]):
    if idx % 2 == 1:
        memory.save_context(
            inputs={"human" : talk['content']},
            outputs={'ai' : tmp }
        )
    else:
        tmp = talk['content']

/tmp/ipykernel_1293/1136332205.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


[SystemMessage(content="The human apologizes, admitting they were momentarily swayed and acted selfishly, causing hurt. The AI asks why the human acted that way. The human expresses gratitude for the AI's honesty, acknowledging their own struggles and complex feelings. They feel somewhat relieved by the AI's sincere apology and hope for more open communication to prevent future issues. The AI thanks the human for their words, reflects on their actions, and promises to avoid causing hurt again, apologizing once more.", additional_kwargs={}, response_metadata={})]


/tmp/ipykernel_1293/1136332205.py:19: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


In [11]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 23.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
